In [1]:
import pandas as pd
import json
from collections import defaultdict
import requests
import os


In [16]:
data = pd.read_clipboard(sep='\t')

clean_booking_id = data['Booking ID'].str.replace("Booking-DHL ID : ", "").str.split(" , ", expand=False)
clean_booking_id = pd.DataFrame(clean_booking_id)

stockoutid = data['Stock Out (ID)']
branchid = data['Branch (ID)']
id_concat = pd.DataFrame(stockoutid + "-" + branchid).rename(columns={0: "PHYID"}, inplace=False).PHYID.str.replace(",","")

frames = [id_concat, clean_booking_id]
new_data = pd.concat(frames, axis=1)
new_data = new_data.dropna().explode(column='Booking ID').set_index(keys='Booking ID')

In [17]:
nested_dict = lambda: defaultdict(dict)
nested = nested_dict()

for row in new_data.itertuples():
    nested[row.Index] = {'PHYID' : row.PHYID}

with open("shipment_data.json", "w+") as jsonfile:
    json.dump(nested, jsonfile, indent=3)
    #write to my firebase

